In [1]:
import lonboard
from lonboard import viz
import pyrosm
from pyrosm import get_data, OSM
from pyrosm.data import sources
import pandas as pd
import geopandas as gpd
import ast
import json
import re
import osmium as o
from pyogrio import read_info, read_dataframe

In [ ]:
# From inspecting pbf files
layers = ['points', 'lines', 'multilinestrings', 'multipolygons']

gdf_list = []

# Iterate through the layers and read each one
for layer in layers:
    try:
        # Read the layer from the PBF file
        gdf = gpd.read_file("../data/raw/output.pbf", engine="pyogrio", layer=layer)
        
        # Add a new column to indicate the layer
        gdf['layer'] = layer
        
        # Append the GeoDataFrame to the list
        gdf_list.append(gdf)

        # Optionally print the first few rows of the GeoDataFrame
        print(f"Layer: {layer}, Number of features: {len(gdf)}")

        # Export the layer as a separate .geojson, to later add as tile layers in tippecanoe

        if len(gdf) > 0:
            export = gdf[["osm_id", "geometry"]]
            export.to_file(f"../data/raw/geojsons/{layer}.geojson", driver="GeoJSON")
        else:
            print(f"Nothing to write to {layer}.geojson. Skipping...")
    except Exception as e:
        print(f"Failed to read layer {layer}: {e}")

# Concatenate all GeoDataFrames in the list into a single GeoDataFrame
combined_gdf = gpd.GeoDataFrame(pd.concat(gdf_list, ignore_index=True))

combined_gdf_min = combined_gdf[["osm_id", "geometry"]]
# combined_gdf_min.to_file("../data/water_plus.geojson", driver="GeoJSON")

Layer: points, Number of features: 15814
Layer: lines, Number of features: 242
Layer: multilinestrings, Number of features: 0
Nothing to write to multilinestrings.geojson. Skipping...
Layer: multipolygons, Number of features: 828


In [5]:
viz(combined_gdf_min)

Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…

In [ ]:
# Help for the engine: https://github.com/geopandas/geopandas/discussions/3015

water = gpd.read_file("../data/raw/drinking_water.osm.pbf", engine="pyogrio", layer = "points")
toilets = gpd.read_file("../data/raw/toilets.osm.pbf", engine="pyogrio", layer="points")

In [ ]:
print(len(water), "drinking water POIs and", len(toilets), "toilet POIs")

In [ ]:
water = water[["osm_id", "name", "geometry", "other_tags"]]
toilets = toilets[["osm_id", "name", "geometry", "other_tags"]]

In [ ]:
viz([toilets, water])

In [ ]:
water_min = water[["osm_id", "geometry"]]
# water_min.to_file("../data/water.geojson", driver="GeoJSON")

In [ ]:
toilets_min = toilets[["osm_id", "geometry"]]
# toilets_min.to_file("../data/toilets.geojson", driver="GeoJSON")